In [1]:
## Get Historical Datas of ETF or STK from YHfinance

import yfinance as yf
import pandas as pd

#-----DEM-Friendly list-----
# 'XLV' : Health care ETF
# 'TAN' : Solar Energy ETF
#-----REP-Friendly list-----
# 'XLE' : Traditional Energy ETF
# 'ITA' : Aerospace & Defense ETF
#-----Comparison list-----
# '^IXIC' : Nasdaq index

STK=['XLV','TAN','XLE','ITA','^IXIC']


periods = [
    ('2008-08-04','2008-11-05'),
    ('2012-08-06','2012-11-07'),
    ('2016-08-08','2016-11-09'),
    ('2020-08-03','2020-11-04'),
    ('2024-08-05','2024-11-06')
]

hist_data = {}
for ticker in STK:
    tk = yf.Ticker(ticker)
    hist_data[ticker] = {}
    for start, end in periods:
        df = tk.history(start=start, end=end)
        hist_data[ticker][(start, end)] = df
        print(f"Fetched {ticker} from {start} to {end}, {len(df)} rows")


Fetched XLV from 2008-08-04 to 2008-11-05, 66 rows
Fetched XLV from 2012-08-06 to 2012-11-07, 64 rows
Fetched XLV from 2016-08-08 to 2016-11-09, 66 rows
Fetched XLV from 2020-08-03 to 2020-11-04, 66 rows
Fetched XLV from 2024-08-05 to 2024-11-06, 66 rows
Fetched TAN from 2008-08-04 to 2008-11-05, 66 rows
Fetched TAN from 2012-08-06 to 2012-11-07, 64 rows
Fetched TAN from 2016-08-08 to 2016-11-09, 66 rows
Fetched TAN from 2020-08-03 to 2020-11-04, 66 rows
Fetched TAN from 2024-08-05 to 2024-11-06, 66 rows
Fetched XLE from 2008-08-04 to 2008-11-05, 66 rows
Fetched XLE from 2012-08-06 to 2012-11-07, 64 rows
Fetched XLE from 2016-08-08 to 2016-11-09, 66 rows
Fetched XLE from 2020-08-03 to 2020-11-04, 66 rows
Fetched XLE from 2024-08-05 to 2024-11-06, 66 rows
Fetched ITA from 2008-08-04 to 2008-11-05, 66 rows
Fetched ITA from 2012-08-06 to 2012-11-07, 64 rows
Fetched ITA from 2016-08-08 to 2016-11-09, 66 rows
Fetched ITA from 2020-08-03 to 2020-11-04, 66 rows
Fetched ITA from 2024-08-05 to 

In [2]:
##Election Neutral Strategy
# Buy each Party's portfolio 50:50 3months before the election and Sell a day after election
import pandas as pd

DEM = ['XLV','TAN']   
REP = ['XLE','ITA']    
alloc = 25.0   # 25% for each Stocks

ALL = DEM + REP
def build_simple_portfolio(hist_data, dem, rep, period, alloc=25.0):
    """
    Buy each ticker at the first trading day's OPEN, 
    then hold until the last trading day's OPEN.
    Return: DataFrame with daily portfolio value.
    """
    legs = []
    for t in dem + rep:
        df = hist_data[t].get(period)
        if df is None or df.empty:
            return None
        entry_day = df.index[0]
        entry_open = df.loc[entry_day, 'Open']
        shares = alloc / float(entry_open)    
        series = shares * df['Open']
        series.name = t
        legs.append(series)
    dfp = pd.concat(legs, axis=1, join='inner')
    dfp['Portfolio'] = dfp.sum(axis=1)
    return dfp


results = {}
for p in periods:
    dfp = build_simple_portfolio(hist_data, DEM, REP, p)
    if dfp is not None:
        results[p] = dfp
        start_val = dfp['Portfolio'].iloc[0]
        end_val   = dfp['Portfolio'].iloc[-1]
        ret = (end_val/start_val - 1.0) * 100
        print(f"{p}: Return {ret:.2f}%")

# results[(start,end)] → DataFrame with columns [DEM, REP, Portfolio]


('2008-08-04', '2008-11-05'): Return -27.85%
('2012-08-06', '2012-11-07'): Return 1.69%
('2016-08-08', '2016-11-09'): Return -5.79%
('2020-08-03', '2020-11-04'): Return 9.73%
('2024-08-05', '2024-11-06'): Return 5.36%


In [3]:
## Poll-based Election neutal strategy
# prepare dataset

import pandas as pd
polldata=pd.read_csv("polldata.csv",parse_dates=["modeldate"])

# Top2candidate
top2poll = (
    polldata
    .groupby(['modeldate'], group_keys=False)  
    .apply(lambda g: g.nlargest(2, 'pct_trend_adjusted'))  
)

# Candidate - Party Mapping
CANDIDATE_TO_PARTY = {
    # DEMOCRAT
    'Al Gore': 'DEM',
    'John Kerry': 'DEM',
    'Barack Obama': 'DEM',
    'Hillary Rodham Clinton': 'DEM',
    'Joseph R. Biden Jr.': 'DEM',
    'Harris': 'DEM',
    # REPUBLICAN
    'George W. Bush': 'REP',
    'John McCain': 'REP',
    'Mitt Romney': 'REP',
    'Donald Trump': 'REP',
    'Trump': 'REP'
}

def candidate_to_party(name: str) -> str:
    return CANDIDATE_TO_PARTY.get(name, 'OTHER') 


top2poll = top2poll.copy()
top2poll['party'] = top2poll['candidate_name'].map(candidate_to_party)

print(top2poll.head())

#leader_gap: Get a poll gap between two candidates and the leader
leader_gap = (
    top2poll
    .groupby(['modeldate','cycle'])
    .apply(lambda g: pd.Series({
        'leader': g.loc[g['pct_trend_adjusted'].idxmax(), 'candidate_name'],
        'gap_pp': g['pct_trend_adjusted'].max() - g['pct_trend_adjusted'].min(),
        'party' : g.loc[g['pct_trend_adjusted'].idxmax(), 'party']
    }))
    .reset_index()
)


/var/folders/cw/gsf83db94mbbq40jtb_6bm240000gn/T/ipykernel_5365/1748123522.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  polldata


      Unnamed: 0  cycle     state  modeldate        candidate_name  \
8320      216842   1968  National 1968-04-09      Richard M. Nixon   
8319      216841   1968  National 1968-04-09  Hubert Humphrey, Jr.   
8323      216845   1968  National 1968-04-10      Richard M. Nixon   
8322      216844   1968  National 1968-04-10  Hubert Humphrey, Jr.   
8326      216848   1968  National 1968-04-11      Richard M. Nixon   

      pct_trend_adjusted  party  
8320            47.40974  OTHER  
8319            35.15829  OTHER  
8323            47.40974  OTHER  
8322            35.15829  OTHER  
8326            47.40974  OTHER  


/var/folders/cw/gsf83db94mbbq40jtb_6bm240000gn/T/ipykernel_5365/1748123522.py:42: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top2poll


In [4]:
## Poll-based Election neutal strategy

from typing import Optional, Tuple, List, Dict
import pandas as pd
import numpy as np

# NOTE: Refactored helper utilities to reduce duplication and cache trading-day lookups.
# ---------- helpers function ----------

#convert to datetime 
def ensure_naive_dtindex(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    idx = pd.to_datetime(out.index, errors='coerce')
    if getattr(idx, 'tz', None) is not None:
        idx = idx.tz_convert(None)
    out.index = idx
    return out.sort_index()

#Searcing for Trading day  after poll
def first_trading_on_or_after(cal_day: pd.Timestamp, trading_idx: pd.DatetimeIndex) -> Optional[pd.Timestamp]:
    pos = trading_idx.searchsorted(pd.Timestamp(cal_day), side='left')
    return trading_idx[pos] if pos < len(trading_idx) else None

def build_period_prices(
    hist_data: Dict[str, Dict[Tuple[str, str], pd.DataFrame]],
    tickers: List[str],
    period: Tuple[str, str]
) -> Tuple[Optional[pd.DataFrame], Optional[pd.DataFrame], Optional[pd.DatetimeIndex]]:
    price_frames = {'Open': [], 'Close': []}
    for t in tickers:
        df = hist_data.get(t, {}).get(period)
        if df is None or df.empty:
            return None, None, None
        df = ensure_naive_dtindex(df)
        if not {'Open', 'Close'}.issubset(df.columns):
            return None, None, None
        price_frames['Open'].append(df[['Open']].rename(columns={'Open': t}))
        price_frames['Close'].append(df[['Close']].rename(columns={'Close': t}))
    px_open = pd.concat(price_frames['Open'], axis=1, join='inner').sort_index()
    px_close = pd.concat(price_frames['Close'], axis=1, join='inner').sort_index()
    idx = px_open.index.intersection(px_close.index)
    return px_open.loc[idx], px_close.loc[idx], idx

def prep_leader_gap(leader_gap: pd.DataFrame) -> pd.DataFrame:
    """Expect columns: modeldate, leader, gap_pp, party."""
    lg = leader_gap.copy()
    need = {'modeldate', 'leader', 'gap_pp', 'party'}
    missing = need - set(lg.columns)
    if missing:
        raise ValueError(f"leader_gap missing columns: {missing}")
    lg['modeldate'] = pd.to_datetime(lg['modeldate'], errors='coerce')
    lg = lg.dropna(subset=['modeldate', 'gap_pp', 'party'])
    lg = lg.set_index('modeldate').sort_index()
    if getattr(lg.index, 'tz', None) is not None:
        lg.index = lg.index.tz_convert(None)
    lg.index = lg.index.normalize()
    lg = lg[~lg.index.duplicated(keep='last')]
    return lg

# ---------- core strategy ----------
def run_poll_gap_strategy_from_leadergap(
    hist_data: Dict[str, Dict[Tuple[str, str], pd.DataFrame]],
    periods: List[Tuple[str, str]],
    leader_gap: pd.DataFrame,                    # <- your table with leader,gap_pp,party
    election_dates: Dict[Tuple[str,str], str],   # period -> 'YYYY-MM-DD'
    dem: List[str],                              # e.g. ['XLV','TAN']
    rep: List[str],                              # e.g. ['XLE','ITA']
    per_leg_allocation: float = 25.0,
    gap_threshold: float = 5.0
) -> Tuple[pd.DataFrame, pd.DataFrame, Dict[str, float]]:
    """
    Rules:
      1) gap ≤ 5  → next OPEN: buy FULL (DEM+REP 50:50)
         gap > 5  → next OPEN: buy leader-party only
      2) If FULL and gap > 5 → next OPEN: sell losing party only
         If LEADER-ONLY and gap ≤ 5 → next OPEN: sell ALL, then buy FULL
      3) Final exit by election-eve gap:
         - eve gap ≤ 5 → exit at election+1 calendar day CLOSE
         - eve gap > 5 → exit at ELECTION DAY OPEN
      * Ignore Fri(4) & Sat(5) poll days; use only same-day poll (no ffill).
    """
    ALL = list(dem) + list(rep)
    side_members = {'DEM': list(dem), 'REP': list(rep)}
    # NOTE: Shared ticker groups for DEM/REP to avoid repeated list literals.
    lg_all = prep_leader_gap(leader_gap)

    action_logs: List[Dict[str, object]] = []
    per_period:  List[Dict[str, object]] = []

    def buy_side(side: str, nd: pd.Timestamp, px_open: pd.DataFrame,
                 shares: Dict[str, float], cash: float) -> float:
        for t in side_members[side]:
            price = float(px_open.loc[nd, t])
            qty = per_leg_allocation / price
            shares[t] = shares.get(t, 0.0) + qty
            cash -= per_leg_allocation
        return cash

    # NOTE: Helper to buy both sides in one call (new).
    def buy_full(nd: pd.Timestamp, px_open: pd.DataFrame,
                 shares: Dict[str, float], cash: float) -> float:
        for side in ('DEM', 'REP'):
            cash = buy_side(side, nd, px_open, shares, cash)
        return cash

    def sell_side(side: str, nd: pd.Timestamp, price_kind: str,
                  px_open: pd.DataFrame, px_close: pd.DataFrame,
                  shares: Dict[str, float], cash: float) -> float:
        for t in side_members[side]:
            qty = shares.get(t, 0.0)
            if qty <= 0.0:
                continue
            price_frame = px_open if price_kind == 'OPEN' else px_close
            price = float(price_frame.loc[nd, t])
            cash += qty * price
            shares[t] = 0.0
        return cash

    def sell_all(nd: pd.Timestamp, price_kind: str,
                 px_open: pd.DataFrame, px_close: pd.DataFrame,
                 shares: Dict[str, float], cash: float) -> float:
        for side in ('DEM', 'REP'):
            cash = sell_side(side, nd, price_kind, px_open, px_close, shares, cash)
        return cash

    # NOTE: Helper to flip exposure without duplicate sell/buy code.
    def rotate_position(from_side: str, to_side: str, nd: pd.Timestamp,
                        px_open: pd.DataFrame, px_close: pd.DataFrame,
                        shares: Dict[str, float], cash: float) -> float:
        cash = sell_side(from_side, nd, 'OPEN', px_open, px_close, shares, cash)
        return buy_side(to_side, nd, px_open, shares, cash)

    for period in periods:
        start_str, end_str = period
        year = pd.Timestamp(start_str).year

        px_open, px_close, t_idx = build_period_prices(hist_data, ALL, period)
        if t_idx is None or t_idx.empty:
            per_period.append({'year': year, 'period': period,
                               'n_actions': 0, 'mode_end': 'FLAT', 'period_return_pct': np.nan})
            continue

        # calendar & poll signals (exact days only)
        cal_idx = pd.date_range(start_str, end_str, freq='D').normalize()
        g = lg_all.reindex(cal_idx)  # may include NaNs where no poll
        # NOTE: Cached gap/party columns as arrays to cut Series lookups.
        gap_values = g['gap_pp'].to_numpy()
        party_values = g['party'].to_numpy()

        # ignore Fri(4) & Sat(5)
        use_mask = (cal_idx.weekday != 4) & (cal_idx.weekday != 5)

        # election-day exit plan
        e_day_str = election_dates.get(period, None)
        if e_day_str is not None:
            e_day = pd.Timestamp(e_day_str).normalize()
            election_open = first_trading_on_or_after(e_day, t_idx)
            eve = e_day - pd.Timedelta(days=1)
            eve_gap = g.loc[eve, 'gap_pp'] if (eve in g.index and pd.notna(g.loc[eve, 'gap_pp'])) else np.nan
            if pd.notna(eve_gap) and eve_gap <= gap_threshold:
                # exit at election+1 calendar day CLOSE
                day_after = e_day + pd.Timedelta(days=1)
                exit_close_day = first_trading_on_or_after(day_after, px_close.index)
                final_exit = ('CLOSE', exit_close_day if exit_close_day is not None else t_idx[-1])
            else:
                # default (eve gap > 5 OR no poll on eve): exit at ELECTION DAY OPEN
                final_exit = ('OPEN', election_open if election_open is not None else t_idx[0])
        else:
            final_exit = ('CLOSE', t_idx[-1])  # fallback

        # cache next trading day lookups so we do not binary-search repeatedly
        next_open_cache: Dict[pd.Timestamp, Optional[pd.Timestamp]] = {}
        # NOTE: Cache next trading open lookups per calendar day.
        def next_open_after(day: pd.Timestamp) -> Optional[pd.Timestamp]:
            key = pd.Timestamp(day).normalize()
            if key not in next_open_cache:
                next_open_cache[key] = first_trading_on_or_after(key + pd.Timedelta(days=1), t_idx)
            return next_open_cache[key]

        # state
        cash0 = per_leg_allocation * len(ALL)
        cash = cash0
        shares: Dict[str, float] = {t: 0.0 for t in ALL}
        mode = 'FLAT'  # 'FLAT','FULL','DEM_ONLY','REP_ONLY'
        n_actions = 0

        for i, d in enumerate(cal_idx):
            if d > pd.Timestamp(end_str):
                break

            # election liquidation when reached
            kind, when_ts = final_exit
            if when_ts is not None and d >= when_ts.normalize():
                if any(shares[t] > 0 for t in ALL):
                    cash = sell_all(when_ts, kind, px_open, px_close, shares, cash)
                    n_actions += len(ALL)
                    action_logs.append({'year': year, 'date': when_ts, 'action': f'ELECTION_{kind}_LIQ', 'cash': cash})
                    mode = 'FLAT'
                break

            if not use_mask[i]:
                continue

            gap_val = gap_values[i]
            if pd.isna(gap_val):
                continue

            gap_val = float(gap_val)
            leader_party = party_values[i]
            if pd.isna(leader_party):
                leader_party = None

            if gap_val <= gap_threshold:
                # target: FULL
                nd = next_open_after(d)
                if nd is not None:
                    if mode == 'FLAT':
                        cash = buy_full(nd, px_open, shares, cash)
                        mode = 'FULL'; n_actions += 2
                        action_logs.append({'year': year, 'date': nd, 'action': 'BUY_FULL', 'cash': cash})
                    elif mode == 'FULL':
                        pass
                    elif mode in ('DEM_ONLY', 'REP_ONLY'):
                        # sell all then rebuy FULL
                        cash = sell_all(nd, 'OPEN', px_open, px_close, shares, cash)
                        n_actions += len(ALL)
                        action_logs.append({'year': year, 'date': nd, 'action': 'SELL_ALL_FOR_REBAL', 'cash': cash})
                        cash = buy_full(nd, px_open, shares, cash)
                        mode = 'FULL'; n_actions += 2
                        action_logs.append({'year': year, 'date': nd, 'action': 'REBUY_FULL', 'cash': cash})

            else:  # gap > threshold → leader-only
                if leader_party in ('DEM', 'REP'):
                    losing = 'DEM' if leader_party == 'REP' else 'REP'
                    nd = next_open_after(d)
                    if nd is not None:
                        if mode == 'FLAT':
                            cash = buy_side(leader_party, nd, px_open, shares, cash)
                            mode = f'{leader_party}_ONLY'
                            n_actions += len(side_members[leader_party])
                            action_logs.append({'year': year, 'date': nd, 'action': f'BUY_{leader_party}_ONLY', 'cash': cash})
                        elif mode == 'FULL':
                            cash = sell_side(losing, nd, 'OPEN', px_open, px_close, shares, cash)
                            mode = f'{leader_party}_ONLY'
                            n_actions += len(side_members[losing])
                            action_logs.append({'year': year, 'date': nd, 'action': f'SELL_{losing}', 'cash': cash})
                        elif mode == 'DEM_ONLY' and leader_party == 'REP':
                            cash = rotate_position('DEM', 'REP', nd, px_open, px_close, shares, cash)
                            mode = 'REP_ONLY'; n_actions += len(ALL)
                            action_logs.append({'year': year, 'date': nd, 'action': 'ROTATE_TO_REP', 'cash': cash})
                        elif mode == 'REP_ONLY' and leader_party == 'DEM':
                            cash = rotate_position('REP', 'DEM', nd, px_open, px_close, shares, cash)
                            mode = 'DEM_ONLY'; n_actions += len(ALL)
                            action_logs.append({'year': year, 'date': nd, 'action': 'ROTATE_TO_DEM', 'cash': cash})
                # else: unknown party → skip

        # safety: if still holding (election exit didn’t trigger), close at last trading CLOSE
        if any(shares[t] > 0 for t in ALL):
            last_td = t_idx[-1]
            cash = sell_all(last_td, 'CLOSE', px_open, px_close, shares, cash)
            n_actions += len(ALL)
            action_logs.append({'year': year, 'date': last_td, 'action': 'FINAL_LIQ', 'cash': cash})
            mode = 'FLAT'

        period_ret = (cash / cash0 - 1.0) * 100.0
        per_period.append({'year': year, 'period': period,
                           'n_actions': n_actions, 'mode_end': mode, 'period_return_pct': period_ret})

    trades_df = pd.DataFrame(action_logs).sort_values(['year', 'date']) if action_logs else pd.DataFrame(columns=['year', 'date', 'action', 'cash'])
    summary_df = pd.DataFrame(per_period).sort_values('year')

    win_rate = (summary_df['period_return_pct'] > 0).mean() * 100.0 if not summary_df.empty else np.nan
    avg_ret = summary_df['period_return_pct'].mean() if not summary_df.empty else np.nan
    tot_comp = ((1.0 + summary_df['period_return_pct'].fillna(0.0) / 100.0).prod() - 1.0) * 100.0 if not summary_df.empty else np.nan
    stats = {'win_rate_pct': float(win_rate) if pd.notna(win_rate) else np.nan,
             'avg_return_pct': float(avg_ret) if pd.notna(avg_ret) else np.nan,
             'total_compounded_pct': float(tot_comp) if pd.notna(tot_comp) else np.nan}
    return trades_df, summary_df, stats


In [5]:
DEM = ['XLV','TAN']
REP = ['XLE','ITA']

# period -> Election Day 매핑
election_dates = {
    ('2000-08-07','2000-11-08'): '2000-11-07',
    ('2004-08-02','2004-11-03'): '2004-11-02',
    ('2008-08-04','2008-11-05'): '2008-11-04',
    ('2012-08-06','2012-11-07'): '2012-11-06',
    ('2016-08-08','2016-11-09'): '2016-11-08',
    ('2020-08-03','2020-11-04'): '2020-11-03',
    ('2024-08-05','2024-11-06'): '2024-11-05',
}

# leader_gap: columns ['modeldate','leader','gap_pp','party'] 로 이미 준비했다고 했음
trades_df, summary_df, stats = run_poll_gap_strategy_from_leadergap(
    hist_data=hist_data,
    periods=periods,
    leader_gap=leader_gap,
    election_dates=election_dates,
    dem=DEM, rep=REP,
    per_leg_allocation=25.0,
    gap_threshold=5.0
)

print(trades_df[trades_df['year']==2020])
print(summary_df)
print(stats)


    year                date             action        cash
12  2020 2020-08-04 04:00:00       BUY_DEM_ONLY   50.000000
13  2020 2020-11-03 05:00:00  ELECTION_OPEN_LIQ  111.529212
   year                    period  n_actions mode_end  period_return_pct
0  2008  (2008-08-04, 2008-11-05)          8     FLAT         -24.655023
1  2012  (2012-08-06, 2012-11-07)          6     FLAT           1.471285
2  2016  (2016-08-08, 2016-11-09)         20     FLAT          -5.381430
3  2020  (2020-08-03, 2020-11-04)          6     FLAT          11.529212
4  2024  (2024-08-05, 2024-11-06)          6     FLAT           3.766202
{'win_rate_pct': 60.0, 'avg_return_pct': -2.653950712849103, 'total_compounded_pct': -16.282085421941783}


In [6]:
nasdaq_results = []
for start, end in periods:
    df = hist_data['^IXIC'].get((start,end))
    if df is None or df.empty:
        ret = None
    else:
        start_open = df.iloc[0]['Open']
        end_close  = df.iloc[-1]['Close']
        ret = (end_close / start_open - 1.0) * 100
    nasdaq_results.append({'period': (start,end), 'nasdaq_return_pct': ret})

nasdaq_returns_df = pd.DataFrame(nasdaq_results)
print(nasdaq_returns_df)


                     period  nasdaq_return_pct
0  (2008-08-04, 2008-11-05)         -22.930187
1  (2012-08-06, 2012-11-07)           1.133922
2  (2016-08-08, 2016-11-09)          -0.575277
3  (2020-08-03, 2020-11-04)           2.875297
4  (2024-08-05, 2024-11-06)          17.353282
